On stock exchange trading is realised by placing buy and sell orders with given price and quantity. Those orders can be added and/or removed. 

Having the following orders:

| Id  | Order | Type   | Price | Quantity |
|-----|-------|--------|-------|----------|
| 001 | Buy   | Add    | 20.00 | 100      |
| 002 | Sell  | Add    | 25.00 | 200      |
| 003 | Buy   | Add    | 23.00 | 50       |
| 004 | Buy   | Add    | 23.00 | 70       |
| 003 | Buy   | Remove | 23.00 | 50       |
| 005 | Sell  | Add    | 28.00 | 100      |


1. Create a data structure that stores orders
2. Impelement Add and Remove operations
3. After placing an individual order, display buy/sell orders with best price

# Basic version

In [1]:
# create dictionary to store orders, key = Id, values: order, price, quantity
orders = {}

In [2]:
# create functions to impelemnt operations

def add_order(order_id, order_type, price, quantity):
    try:
        key = (order_id, order_type)
        orders[key] = {
            "price": price,
            "quantity": quantity }
        display_best_orders()
    except Exception as e:
        print(f"Error while adding order: {e}")
    
def remove_order(order_id, order_type, price, quantity):
    try:
        key = (order_id, order_type)
        if key in orders:
            del orders[key]
        display_best_orders()
    except Exception as e:
        print(f"Error while removing order: {e}")

def display_best_orders():
    try:
        buy_orders = [order for key, order in orders.items() if key[1] == "Buy"]
        sell_orders = [order for key, order in orders.items() if key[1] == 'Sell']

        best_buy = min(buy_orders, key = lambda x: x['price'], default = None)
        best_sell = max(sell_orders, key = lambda x: x['price'], default = None )

        print("Best Buy Order:", best_buy) 
        print("Best Sell Order:", best_sell)
    except Exception as e:
        print(f"Error while displaying best orders: {e}")

Executing orders

In [3]:
add_order('001', 'Buy', 20.00, 100)

Best Buy Order: {'price': 20.0, 'quantity': 100}
Best Sell Order: None


In [4]:
add_order('002', 'Sell', 25.00, 200)

Best Buy Order: {'price': 20.0, 'quantity': 100}
Best Sell Order: {'price': 25.0, 'quantity': 200}


In [5]:
add_order('003', 'Buy', 23.00, 50)

Best Buy Order: {'price': 20.0, 'quantity': 100}
Best Sell Order: {'price': 25.0, 'quantity': 200}


In [6]:
add_order('004', 'Buy', 23.00, 70)

Best Buy Order: {'price': 20.0, 'quantity': 100}
Best Sell Order: {'price': 25.0, 'quantity': 200}


In [7]:
remove_order('003', 'Buy', 23.00, 50)

Best Buy Order: {'price': 20.0, 'quantity': 100}
Best Sell Order: {'price': 25.0, 'quantity': 200}


In [8]:
add_order('005', 'Sell', 28.00, 100)

Best Buy Order: {'price': 20.0, 'quantity': 100}
Best Sell Order: {'price': 28.0, 'quantity': 100}


This is the basic implementation of the solution to the recruitment task. However, one may be tempted to expand it.

# Expanded version

Let's assume that the orders are stored in a pre-made PostgreSQL database that we have access to, and that the operations performed in Python affect this table.
This is how we could implement better solution:

In [9]:
# relevant imports
import psycopg2
from psycopg2 import OperationalError, DatabaseError

# create the connection with database
def connect_db(dbname, user, password, host, port):
    try:
        conn = psycopg2.connect(
            dbname=dbname, 
            user=user, 
            password=password, 
            host=host, 
            port=port  # make sure to use integer
        )
        print("Connected to the database successfully.")
        return conn
    
    except OperationalError as e:
        print("Failed to cennect to the database:", e)
        return None

In [10]:
# create functions
def add_order_db(conn, order_id, order_type, price, quantity):
    try:
        with conn.cursor() as cur: 
            cur.execute(""" INSERT INTO orders (id, order, price, quantity) VALUES (%s, %s, %s, %s) """, 
                        (order_id, order_type, price, quantity)) 
            conn.commit()

            display_best_orders_db(conn)
    except (OperationalError, DatabaseError, AttributeError) as e:
        print(f"Error while adding order: {e}")
        conn.rollback()

def remove_order_db(conn, order_id, order_type):
    try:
        with conn.cursor() as cur: 
            cur.execute(""" DELETE FROM orders WHERE id = %s AND order = %s """, (order_id, order_type)) 
            conn.commit()

            display_best_orders_db(conn)
    except (OperationalError, DatabaseError, AttributeError) as e:
        print(f"Error while removing order: {e}")
        conn.rollback()

def display_best_orders_db(conn): 
    try:
        with conn.cursor() as cur: 
            # import buy orders
            cur.execute(""" SELECT * FROM orders WHERE order = 'Buy' ORDER BY price ASC LIMIT 1 """)
            best_buy = cur.fetchone() 

            # import sell orders
            cur.execute(""" SELECT * FROM orders WHERE order = 'Sell' ORDER BY price DESC LIMIT 1 """) 
            best_sell = cur.fetchone() 

            print("Best Buy Order:", best_buy) 
            print("Best Sell Order:", best_sell)
    except (OperationalError, DatabaseError, AttributeError) as e: 
        print(f"Error while displaying best orders: {e}")

Exceuting orders

In [11]:
# connection with db
conn = connect_db("dbname", "user", "password", "host", "port")  # fill with proper values

#conn vaidation
if conn is None: 
    print("No connection to the database.")

else:
    try:
        # orders
        add_order_db(conn, '001', 'Buy', 20.00, 100)
        add_order_db(conn, '002', 'Sell', 25.00, 200)
        add_order_db(conn, '003', 'Buy', 23.00, 50)
        add_order_db(conn, '004', 'Buy', 23.00, 70)
        remove_order_db(conn, '003', 'Buy')
        add_order_db(conn, '005', 'Sell', 28.00, 100)

        # always remember to close the connection
        conn.close()
    except Exception as e:
        print("Error while running orders:", e)

Failed to cennect to the database: invalid integer value "port" for connection option "port"

No connection to the database.


# Further improvement suggestions

The service could be improved by being able to delete parts of an order instead of just the whole thing. The remove_orders() function in the basic implementation could check not only the key but also the quantity and reduce the quantity in the dictionary when deleting part of the order.

In the database connected version, the remove_orders_db() function would also need to be modified to check the quantity in the table and perform a record update in the case of a partial delete, or delete the entire record in the case of a full delete.